In [1]:
from rsatoolbox import vis
from rsatoolbox import rdm
import rsatoolbox
import rsatoolbox.data as rsd 
import rsatoolbox.rdm as rsr

import numpy as np
import os
import inspect
import scipy.io
from collections import defaultdict
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib
from PIL import Image
import warnings


from sklearn.decomposition import IncrementalPCA
from sklearn.preprocessing import StandardScaler
from sklearn import random_projection

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split, Subset
import torchvision
import torchvision.models as models
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchvision import transforms

from util_function import ImageDataset, get_roi_mapping, seed_everything, image_visualization_transform

In [2]:
seed_everything()

True


In [3]:
data_dir = '/home/yuchen/human/tutorial'
parent_submission_dir = '/home/yuchen/psy221f_project'

batch_size = 32
input_img_dim = (64,64)

device = 'cuda'

subj = 1
roi = ["V1v", "V1d", "V2v", "V2d", "hV4",
       "EBA", "FBA-1", "FBA-2",
       "OFA", "FFA-1", "FFA-2",
       "OPA", "PPA", "RSC","OWFA", "VWFA-1", "VWFA-2", ]

freeze_weights = True  # if True, freeze the visual encoder's weights
subset_training_data = False  # if True, only select a small proportion of the data for training

In [4]:
device = torch.device(device)


class argObj:
  def __init__(self, data_dir, parent_submission_dir, subj):

    self.subj = format(subj, '02')
    self.data_dir = os.path.join(data_dir, 'subj'+self.subj)
    self.parent_submission_dir = parent_submission_dir
    self.subject_submission_dir = os.path.join(self.parent_submission_dir,
        'subj'+self.subj)


args = argObj(data_dir, parent_submission_dir, subj)

train_img_dir  = os.path.join(args.data_dir, 'training_split', 'training_images')

train_img_list = os.listdir(train_img_dir)
train_img_list.sort()

num_train = int(np.round(len(train_img_list) / 100 * 90))
idxs = np.arange(len(train_img_list))
np.random.shuffle(idxs)
idxs_train, idxs_val = idxs[:num_train], idxs[num_train:]

print('Training stimulus images: ' + format(len(idxs_train)))
print('\nValidation stimulus images: ' + format(len(idxs_val)))

train_imgs_paths = sorted(list(Path(train_img_dir).iterdir()))


Training stimulus images: 8857

Validation stimulus images: 984


In [5]:
# model = models.vgg16_bn(weights=models.VGG16_BN_Weights.IMAGENET1K_V1).to('cuda')
# model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1).to('cuda')
# model = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1).to('cuda')
model = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1).to('cuda')
# # model = models.densenet161(weights=models.DenseNet161_Weights.IMAGENET1K_V1).to('cuda')
# model = models.squeezenet1_0(weights=models.SqueezeNet1_0_Weights.IMAGENET1K_V1).to('cuda')
# model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1).to('cuda')
# model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1).to('cuda')




for _, params in model.named_parameters():
    params.require_grad=False

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [6]:

data_dir = args.data_dir
# idxs = idxs_val
idxs = idxs_train[:1000]


train_list = np.array(train_img_list)[idxs]
# train_list = np.array(train_img_list)


In [7]:
# import torch
# from torchvision import transforms
# from torchvision.models import resnet50
# import torch.nn as nn
# from torch.utils.data import DataLoader
# from collections import OrderedDict

# def load_backbone_feature_extractor(pretrained_path=""):
#     """Loads a pre-trained feature extractor.

#     Arguments
#     ---------
#         pretrained_path: str, optional
#             Path to the feature extractor's weights.

#     Returns
#     -------
#     A Pytorch network initialized with pre-trained weights.

#     """
#     net = resnet50(weights=None if pretrained_path == "" else "DEFAULT").cuda()

#     if pretrained_path != "":
#         print("Loading pretrained weights from: ", pretrained_path)

#         # original saved file with DataParallel
#         state_dict = torch.load(pretrained_path)

#         # create new OrderedDict that does not contain `module.`
#         new_state_dict = OrderedDict()
#         for k, v in state_dict.items():
#             if "module." in k:
#                 name = k[7:]  # remove `module.`
#             elif "_feature_blocks." in k:
#                 name = k.replace("_feature_blocks.", "")
#             else:
#                 name = k
#             if name in net.state_dict().keys():
#                 new_state_dict[name] = v
#             else:
#                 print("key ", name, " not in dict")

#         for key in net.state_dict().keys():
#             if key not in new_state_dict.keys():
#                 print("Network key ", key, " not in dict to load")

#         net.load_state_dict(new_state_dict, strict=False)

#     return net

# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# model = load_backbone_feature_extractor('/home/yuchen/human/swav_feature_extractor/feature_extractor/swav_800ep_pretrain.pth.tar')

# # model.fc = nn.Linear(2048, 28835)

# for name, params in model.named_parameters():
#     params.requires_grad = False
    
# # sample_input = img_transforms(torch.rand(8, 3, 224, 224)).cuda()
# # _ = model(sample_input)

In [8]:
n_components = 5920

In [9]:
ls /home/yuchen/model_rdm/

alexnet/          mobilenet_v2_new/         resnet50_swav_new/
alexnet_new/      resnet18/                 resnet_swav/
densenet161/      resnet18_new/             squeezenet1_0/
inceptionv3/      resnet50/                 squeezenet1_0_new/
inceptionv3_new/  resnet50_new/             vgg16/
mobilenet_v2/     resnet50_swav_allimages/  vgg16_new/


In [10]:
mkdir /home/yuchen/model_rdm/inceptionv3_new/

mkdir: cannot create directory ‘/home/yuchen/model_rdm/inceptionv3_new/’: File exists


In [ ]:
performance_map = {}
model_layer_dir = '/home/yuchen/model_rdm/inceptionv3_new/'

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    for layer_idx, current_feature_layer in enumerate(get_graph_node_names(model)[0][1:]):
        
        model_layer = current_feature_layer
        
        if all(x not in current_feature_layer for x in ['unsqueeze', 'mul', 'add','getitem','cat']):
            if -1 != 0: 
                model_layer = current_feature_layer
                print(model_layer)
                    
                feature_extractor = create_feature_extractor(model, return_nodes=[model_layer]).to('cuda')

                # feature_map_lst = np.array([])
                feature_map_lst = []
                # feature_map_lst_100 = []
                
                with torch.no_grad():
                    for idx, train_img_dir_individual in enumerate(train_list):
                        image = Image.open(f'{args.data_dir}/training_split/training_images/{train_img_dir_individual}').convert('RGB')

                        input_tensor = transform(image).to('cuda')
                        
                        input_tensor = torch.unsqueeze(input_tensor,0)

                        feature_map = feature_extractor(input_tensor)[model_layer]
                        feature_map = feature_map.cpu().detach().numpy()
                        feature_map = feature_map.reshape(-1)
                        feature_map_lst.append(feature_map)
                        
                feature_map_lst = np.array(feature_map_lst)
                    
                #     if len(feature_map) > 5920:
                #         feature_map_lst_100.append(feature_map)
                #         if len(feature_map_lst_100) == 100:
                #             feature_map_lst_100 = np.array(feature_map_lst_100)
                #             transformer = random_projection.SparseRandomProjection(n_components=n_components)
                #             x_new = transformer.fit_transform(feature_map_lst_100)
                #             feature_map_lst = np.vstack([feature_map_lst, x_new]) if feature_map_lst.size else x_new
                #             feature_map_lst_100 = []
                #     else: feature_map_lst.append(feature_map)
                    
                # if len(feature_map) > 5920:
                #     feature_map_lst_100 = np.array(feature_map_lst_100)
                #     transformer = random_projection.SparseRandomProjection(n_components=n_components)
                #     x_new = transformer.fit_transform(feature_map_lst_100)
                #     feature_map_lst = np.vstack([feature_map_lst, x_new]) if feature_map_lst.size else x_new
                # else: feature_map_lst = np.array(feature_map_lst)
                    
                measurements = feature_map_lst
                nCond = measurements.shape[0]
                nVox = measurements.shape[1]

                des = {'session': 1, 'subj': 1}
                obs_des = {'conds': np.array(['cond_%02d' % x for x in np.arange(nCond)])}
                chn_des = {'voxels': np.array(['voxel_' + str(x) for x in np.arange(nVox)])}
                data = rsd.Dataset(measurements=measurements,
                                    descriptors=des,
                                    obs_descriptors=obs_des,
                                    channel_descriptors=chn_des)

                RDM_euc = rsr.calc_rdm(data, descriptor='conds')
                dist_matrix_model = RDM_euc.get_matrices()
                    
                np.save(f'{model_layer_dir}{str(layer_idx)}_{model_layer}.npy', dist_matrix_model)
        
    # correlations = np.mean(rsatoolbox.rdm.compare_correlation(dist_matrix_model, dist_matrix_brain))
    
    # performance_map[current_feature_layer] = correlations
    
    # print(correlations,end='\n\n')

Conv2d_1a_3x3.conv
Conv2d_1a_3x3.bn
Conv2d_1a_3x3.relu
Conv2d_2a_3x3.conv
Conv2d_2a_3x3.bn
Conv2d_2a_3x3.relu
Conv2d_2b_3x3.conv
Conv2d_2b_3x3.bn
Conv2d_2b_3x3.relu
maxpool1
Conv2d_3b_1x1.conv
Conv2d_3b_1x1.bn
Conv2d_3b_1x1.relu
Conv2d_4a_3x3.conv
Conv2d_4a_3x3.bn
Conv2d_4a_3x3.relu
maxpool2
Mixed_5b.branch1x1.conv
Mixed_5b.branch1x1.bn
Mixed_5b.branch1x1.relu
Mixed_5b.branch5x5_1.conv
Mixed_5b.branch5x5_1.bn
Mixed_5b.branch5x5_1.relu
Mixed_5b.branch5x5_2.conv
Mixed_5b.branch5x5_2.bn
Mixed_5b.branch5x5_2.relu
Mixed_5b.branch3x3dbl_1.conv
Mixed_5b.branch3x3dbl_1.bn
Mixed_5b.branch3x3dbl_1.relu
Mixed_5b.branch3x3dbl_2.conv
Mixed_5b.branch3x3dbl_2.bn
Mixed_5b.branch3x3dbl_2.relu
Mixed_5b.branch3x3dbl_3.conv
Mixed_5b.branch3x3dbl_3.bn
Mixed_5b.branch3x3dbl_3.relu
Mixed_5b.avg_pool2d
Mixed_5b.branch_pool.conv
Mixed_5b.branch_pool.bn
Mixed_5b.branch_pool.relu
Mixed_5c.branch1x1.conv
Mixed_5c.branch1x1.bn
Mixed_5c.branch1x1.relu
Mixed_5c.branch5x5_1.conv
Mixed_5c.branch5x5_1.bn
Mixed_5c.branc